In [ ]:
import pandas as pd


data = pd.read_csv("Data/PhilipTestEU.csv")

data['Department'] = data['Department'].replace(str(data.iloc[22,1]), 'Language Centre')
data['Department'] = data['Department'].replace(str(data.iloc[117,1]), 'Psychological and Behavioural Science')
data['Department'] = data['Department'].replace(str(data.iloc[155,1]), 'Philosophy Logic and Scientific Method')
data['Department'] = data['Department'].replace(str(data.iloc[254,1]), 'Geography and Environment')
data['Department'] = data['Department'].replace(str(data.iloc[23,1]), 'LSE Law School')


data
# print(data.iloc[254,1])



def load_and_process_files(output_file_path, eu_file_path):
    # Load the datasets
    file1 = pd.read_csv(output_file_path)
    file2 = pd.read_csv(eu_file_path)
    
    # Clean department names in both datasets
    file1['Department'] = file1['Department'].str.replace('Department of ', '').str.strip().str.title()
    file2['Department'] = file2['Department'].str.strip().str.title()

    # Additional cleaning for file2, correcting known inconsistencies
    department_mapping = {
        "Anthropology  Anthropology": "Anthropology",
        "Economic  History": "Economic History",
        "Economic Hi..": "Economic History",
        "Finance  Finance": "Finance",
        "Government  Government": "Government",
        "International  History": "International History",
        "International  Relations": "International Relations",
        "Management  Management": "Management",
        "Philosophy, ..  Psychologic..": "Philosophy Logic And Scientific Method",
        "School Of Public Policy": "Public Policy",
        "Social Policy  Social Policy": "Social Policy",
        "Sociology  Sociology": "Sociology"
    }
    file2['Department'] = file2['Department'].replace(department_mapping)
    
    # Calculate median salary and classify departments
    median_salaries = file1.groupby('Department')['Median Salary'].median()
    median_salary_threshold = median_salaries.median()
    department_salary_classification = median_salaries.apply(
        lambda x: 'High' if x >= median_salary_threshold else 'Low'
    ).reset_index()
    department_salary_classification.columns = ['Department', 'Salary Group']
    
    # Merge salary classification with application data
    final_merged_data = pd.merge(file2, department_salary_classification, on='Department', how='left')
    
    # Save the final merged data to a new CSV file
    final_merged_data.to_csv('PhilipEUCleaned.csv', index=False)
    
    return final_merged_data

# Example usage
output_file_path = 'Data/PhilipOutput.csv'  # Path to the output file
eu_file_path = 'PhilipEU.csv'          # Path to the EU file
cleaned_data = load_and_process_files(output_file_path, eu_file_path)
print(cleaned_data.head())
